### 구동환경
azure notebook,  python3.8-pytorch and Tensorflow 

In [ ]:
# 필요한 라이브러리 설치
!pip install torch torchvision torchaudio matplotlib numpy


In [ ]:
# PyTorch 및 관련 라이브러리 불러오기
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import os
from PIL import Image                           # 이미지 파일 로드를 위한 라이브러리



### PyTorch 및 관련 라이브러리 설명
1. **torch** (PyTorch 핵심 라이브러리)
    
    PyTorch의 기본 기능을 제공하는 라이브러리
    텐서 연산(Tensor Operations), 자동 미분(Autograd) 기능 포함


2. **torchvision** (PyTorch 컴퓨터 비전 라이브러리)
    
    이미지 데이터 처리 및 사전 훈련된 모델 사용을 위한 라이브러리
    
    주요 기능:
    torchvision.datasets → 데이터셋 다운로드 및 로드
    torchvision.transforms → 이미지 변환 (크기 조정, 정규화 등)
    torchvision.models → 사전 훈련된 CNN 모델 (ResNet, VGG 등)


3. **torchvision.transforms** (이미지 변환 및 전처리)
    
    이미지 데이터를 전처리(Preprocessing) 하기 위한 모듈
    크기 조정, 정규화, Tensor 변환 등 수행 가능


4. **torch.nn** (신경망 레이어 및 손실 함수)
    
    신경망 모델을 구성하는 레이어 및 손실 함수 제공


5. **torch.optim** (최적화 알고리즘)
    
    SGD, Adam 등 다양한 최적화 알고리즘 포함

6. **torch.utils.data.DataLoader** (데이터 로더)
    
    데이터셋을 배치 단위로 나누어 모델 학습에 사용하도록 하는 도구
    
    **shuffle=True** → 데이터를 무작위로 섞어 학습
    
    **batch_size=n** → 한 번에 처리할 데이터 개수 설정


7. **torchvision.datasets.ImageFolder** (이미지 폴더에서 데이터셋 불러오기)
   
    폴더 구조를 기반으로 한 이미지 데이터셋 로드
    각 폴더 이름을 **클래스 라벨(Label)**로 자동 인식


8. **matplotlib.pyplot** (데이터 시각화)
    
    그래프, 이미지 등을 시각화하는 라이브러리


9. **os** (파일 및 폴더 조작)
    
    운영체제 관련 기능 제공
    폴더 내 파일 확인, 폴더 생성 등 가능

10. **PIL (Python Imaging Library)**는 이미지 처리를 위한 파이썬 라이브러리
    




# 작업 전 경로 확인
해당 과정은 경로를 정확히 설정 시 생략 가능

In [ ]:
# 현재 작업 폴더 확인 및 데이터 폴더 존재 여부 체크


print("현재 작업 폴더:", os.getcwd())  # 현재 작업 중인 폴더 확인
print("Train 폴더 존재 여부:", os.path.exists("./data_set/sample_data/train")) #해당폴더에 train 데이터 셋이 존재 하면True
print("Test 폴더 존재 여부:", os.path.exists("./data_set/sample_data/test")) #해당폴더에 test 데이터 셋이 존재 하면True


현재 작업 디렉토리: /mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml/code/Users/6a025/cnn_test/data_set
Train 폴더 존재 여부: False
Test 폴더 존재 여부: False


In [ ]:
# CNN 데이터셋이 어디에 있는지 확인
base_paths = [
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml/code/Users/6a025/cnn_test/data_set",
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml/code/Users/6a025/cnn_test",
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml",
    "/home/user/data_set"  # 예상되는 다른 경로에 데이터가 있는 지 표시
]


#base_paths의 주소(폴더)에 있으면 ✅ 데이터셋이 존재하는 폴더: 주소(폴더 위치)표시
#base_paths의 주소(폴더)에 없으면 ❌ 데이터셋 없음: 주소(폴더 위치)표시
for path in base_paths:
    if os.path.exists(path):
        print(f"✅ 데이터셋이 존재하는 폴더: {path}")
    else:
        print(f"❌ 데이터셋 없음: {path}")


✅ 데이터셋이 존재하는 폴더: /mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml/code/Users/6a025/cnn_test/data_set
✅ 데이터셋이 존재하는 폴더: /mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml/code/Users/6a025/cnn_test
✅ 데이터셋이 존재하는 폴더: /mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml
❌ 데이터셋 없음: /home/user/data_set


# 작업 경로 설정 및 데이터 로드



In [ ]:
# 데이터셋이 저장된 폴더 경로 (사용자 환경에 맞게 수정 필요)
train_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml/code/Users/6a025/cnn_test/data_set/sample_data/train"
test_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml/code/Users/6a025/cnn_test/data_set/sample_data/test"

# 1️⃣ 폴더 존재 여부 확인
print(f"Train 폴더 존재 여부: {os.path.exists(train_path)}")  # 학습 데이터 폴더가 존재하는지 확인
print(f"Test 폴더 존재 여부: {os.path.exists(test_path)}")  # 테스트 데이터 폴더가 존재하는지 확인

# 2️⃣ 이미지 변환(전처리) 설정
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 모든 이미지를 128x128 크기로 변환 (모델 입력 크기 통일)
    transforms.ToTensor(),  # 이미지 데이터를 PyTorch 텐서(Tensor) 형식으로 변환
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # RGB 값 정규화 (-1 ~ 1 범위)
])

# 3️⃣ 데이터 로드 (예외 처리 추가)
try:
    train_dataset = datasets.ImageFolder(root=train_path, transform=transform)  # 학습 데이터셋 로드
    test_dataset = datasets.ImageFolder(root=test_path, transform=transform)  # 테스트 데이터셋 로드
    print("✅ 데이터셋 로드 성공!")  # 데이터셋 로드 성공 시 메시지 출력
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없음: {e}")  # 데이터 폴더가 존재하지 않을 경우 예외 처리


Train 폴더 존재 여부: True
Test 폴더 존재 여부: True
✅ 데이터셋 로드 성공!


#### 이미지변환(전처리)
**transforms.Resize((128, 128))**
 →  모든 이미지를 크기 128x128 픽셀로 변환 (CNN 모델 입력 크기를 통일하기 위해 필요)

**transforms.ToTensor()**
 →  이미지를 텐서(Tensor) 형식으로 변환 (PyTorch에서 데이터를 처리하기 위함)

**transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])**
 →  각 픽셀 값을 -1 ~ 1 범위로 정규화(Normalization)
(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]은 RGB 각 채널에 적용)

#### 데이터셋 로드 & 예외 처리
**datasets.ImageFolder(root=train_path, transform=transform)**
 →  train_path 폴더에서 데이터를 불러오고, 위에서 정의한 transform을 적용

**예외 처리 (try-except 문)**
 →  만약 폴더가 존재하지 않으면 FileNotFoundError가 발생하여 "❌ 파일을 찾을 수 없음" 오류 메시지를 출력


# PyTorch의 DataLoader 를 사용하여 미니배치(mini-batch) 형태로 데이터를 로드

In [ ]:
# 1️⃣ DataLoader 설정 (배치 크기: 32)
# 배치 사이즈의 경우 임의로 조정가능
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # 학습 데이터 로더 (랜덤 셔플)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # 테스트 데이터 로더 (순차적 로드)

# 2️⃣ 데이터 확인 (첫 번째 배치만 출력)
for images, labels in train_loader:
    print(f"✅ 데이터 로드 성공! 이미지 크기: {images.shape}, 레이블 크기: {labels.shape}")
    break  # 한 배치만 확인하고 루프 종료


train_dataset과 test_dataset을 배치(batch) 단위로 묶어서 로딩
batch_size=32 → 한 번에 32개의 데이터를 가져옴


**shuffle=True**
train_loader의 경우 데이터를 랜덤으로 섞어서 불러옴 (학습에 중요한 역할)
test_loader는 shuffle=False로 설정하여 데이터 순서를 유지 (테스트 결과 일관성 확보)


#  PyTorch 기반 CNN(합성곱 신경망, Convolutional Neural Network) 모델을 정의 및 음식 이미지 분류 모델

In [ ]:
# 📌 CNN(Convolutional Neural Network) 모델 정의
class FoodCNN(nn.Module):  # nn.Module을 상속받아 새로운 신경망 정의
    def __init__(self, num_classes):
        super(FoodCNN, self).__init__()  # 부모 클래스 초기화

        # 1️⃣ 합성곱(Convolutional) layer 정의
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)  # 입력(3채널) → 32채널 feature map
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)  # 32채널 → 64채널 feature map
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)  # 64채널 → 128채널 featur map

        # 2️⃣ 풀링(Pooling) layer 정의
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # 2x2 크기의 맥스풀링 적용 (특징 맵 크기 절반으로 감소)

        # 3️⃣ 완전 연결(Fully Connected) 레이어 정의
        self.fc1 = nn.Linear(in_features=128 * 16 * 16, out_features=512)  # 128채널 * 16 * 16 크기의 벡터 → 512 뉴런
        self.fc2 = nn.Linear(in_features=512, out_features=num_classes)  # 512 뉴런 → 음식 클래스 개수

    def forward(self, x):
        # 📌 순전파(Forward) 과정 정의
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 → ReLU → MaxPooling
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 → ReLU → MaxPooling
        x = self.pool(F.relu(self.conv3(x)))  # Conv3 → ReLU → MaxPooling

        x = x.view(x.size(0), -1)  # 📌 텐서를 펼쳐서(Flatten) 완전 연결층에 전달
        x = F.relu(self.fc1(x))  # 완전 연결 레이어 1 + ReLU
        x = self.fc2(x)  # 완전 연결 레이어 2 (출력층)

        return x  # 최종 출력값 반환

# 📌 클래스 개수 가져오기
num_classes = len(train_dataset.classes)  # `train_dataset`의 클래스 개수를 가져옴
print(f"✅ 총 {num_classes}개의 음식 클래스가 있습니다.")

# 📌 모델 생성
model = FoodCNN(num_classes)  # FoodCNN 모델을 생성


✅ 총 19개의 음식 클래스가 있습니다.


# 📌 정리

**CNN의 주요 구성 요소**

CNN은 주로 **합성곱 레이어(Convolutional Layer), 활성화 함수(ReLU), 풀링 레이어(Pooling Layer), 완전 연결 레이어(Fully Connected Layer)** 로 구성됩니다.

1. **합성곱(Convolution) 레이어**:
    이미지를 작은 필터(Filter, Kernel) 로 스캔하여 특징을 추출하는 과정

    주요 특징:작은 필터(예: 3×3, 5×5)를 사용해 이미지를 부분적으로 분석
    엣지(edge), 모서리, 텍스처(texture) 등의 패턴을 감지
    stride(이동 크기)와 padding(경계 처리)을 설정하여 출력 크기 조절

2. **활성화 함수 (ReLU)**:
    비선형성을 추가하여 신경망이 복잡한 패턴을 학습할 수 있도록 하는 함수
    ReLU(Rectified Linear Unit) 가 가장 많이 사용

    음수 값은 0으로 변환하고, **양수 값은 그대로 유지**
    선형(Linear) 모델만 사용하면 복잡한 패턴을 학습할 수 없음 -> ReLU를 적용하면 신경망이 더 깊어질 수 있으며, 성능이 향상됨

3. **풀링(Pooling) 레이어**:
    특징 맵의 크기를 줄이고 중요한 정보만 남기는 과정

    주요 기능:연산량 감소 → 모델이 더 가벼워짐
    과적합(Overfitting) 방지 → 특정 특징이 너무 강조되는 것을 방지
    위치 변화에 대한 불변성(Translation Invariance) 유지

4. **완전 연결(Fully Connected, FC) 레이어**:
    CNN의 마지막 단계로, 합성곱 레이어에서 추출된 고유한 특징을 기반으로 최종 분류를 수행하는 역할
    일반적인 다층 퍼셉트론(MLP) 구조와 동일

    Flatten 연산을 사용하여 2D → 1D 벡터 변환 후 사용

# 손실 함수(loss function) 및 옵티마이저(optimizer) 를 설정

In [ ]:
# 📌 GPU 사용 가능하면 GPU로 이동 (그렇지 않으면 CPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # 모델을 해당 장치(GPU 또는 CPU)로 이동

# 📌 손실 함수 정의 (다중 클래스 분류 문제이므로 CrossEntropyLoss 사용)
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류에 적합한 손실 함수

# 📌 옵티마이저 정의 (Adam 사용, 학습률: 0.001) 
optimizer = optim.Adam(model.parameters(), lr=0.001)



**손실 함수(Loss Function) 정의**
    nn.CrossEntropyLoss()
    → 다중 클래스 분류(Multi-class Classification) 에 적합한 손실 함수

    사용 이유:
    클래스 간 확률 분포를 비교하여 예측 성능을 평가,
    Softmax와 Negative Log Likelihood(NLL)를 내부적으로 포함

**옵티마이저(Optimizer)**:

    옵티마이저(Optimizer) 는 신경망이 학습할 때 **가중치(weight)와 편향(bias)**을 조정하는 알고리즘이다.
    즉, 옵티마이저는 **손실 함수(Loss Function) 를 최소화**하는 방향으로 모델을 업데이트하는 역할을 한다.

    역할:
    손실 함수의 기울기(Gradient)를 계산
    backpropagation(역전파)을 통해 손실 함수의 변화량을 구함

    기울기 방향으로 가중치를 조정
    손실이 감소하는 방향으로 가중치를 업데이트

    학습률(learning rate)에 따라 업데이트 크기를 조절
    너무 크면 최적값을 지나칠 수 있고, 너무 작으면 수렴 속도가 느림

# 이미지 학습

In [27]:
num_epochs = 20  # 학습할 에포크 수

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # GPU로 이동

        optimizer.zero_grad()  # 기울기 초기화
        outputs = model(images)  # 모델 예측
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("✅ 학습 완료!")


Epoch [1/20], Loss: 0.1135
Epoch [2/20], Loss: 0.3461
Epoch [3/20], Loss: 0.1933
Epoch [4/20], Loss: 0.0712
Epoch [5/20], Loss: 0.0301
Epoch [6/20], Loss: 0.0099
Epoch [7/20], Loss: 0.0083
Epoch [8/20], Loss: 0.0012
Epoch [9/20], Loss: 0.0013
Epoch [10/20], Loss: 0.0006
Epoch [11/20], Loss: 0.0004
Epoch [12/20], Loss: 0.0003
Epoch [13/20], Loss: 0.0003
Epoch [14/20], Loss: 0.0002
Epoch [15/20], Loss: 0.0002
Epoch [16/20], Loss: 0.0002
Epoch [17/20], Loss: 0.0002
Epoch [18/20], Loss: 0.0002
Epoch [19/20], Loss: 0.0002
Epoch [20/20], Loss: 0.0001
✅ 학습 완료!


1. **model.train() : 모델을 학습 모드로 설정**

    PyTorch 모델에는 학습 모드(train())와 평가 모드(eval()) 가 있음

    model.train()을 호출하면:**Dropout, Batch Normalization** 등 학습 관련 기능 활성화 -> 모델이 가중치 업데이트를 학습 모드로 수행


    왜 필요한가?

    모드에 따라 동작이 달라짐
    train() 모드에서는 **Dropout**이 활성화되고, eval() 모드에서는 비활성화됨

2. **optimizer.zero_grad():** 이전 기울기 초기화

    이전 배치에서 계산된 기울기(Gradient)를 제거

    PyTorch는 **기울기를 누적(accumulate)하는 방식**이므로, 
    zero_grad()를 호출하지 않으면 이전 배치의 기울기가 남아 있음 → 학습이 잘못될 수 있음

3. **outputs = model(images): 순전파(Forward Propagation)**

    images(입력 데이터)를 CNN 모델에 통과시켜 예측값을 생성

    CNN 모델 내부에서: 합성곱(Convolution) → 활성화 함수(ReLU) → 풀링(Pooling) → 완전 연결(FC) 레이어를 거쳐 출력값을 생성


4. **loss = criterion(outputs, labels): 손실 계산**
    
    손실 함수(criterion)를 사용해 예측값(outputs)과 실제값(labels)의 차이를 계산
    
    CrossEntropyLoss()를 사용하므로 다중 클래스 분류에서 확률 분포 기반 손실 계산
    
    loss.item()을 이용하면 손실 값(스칼라 값)으로 변환 가능

5. **loss.backward(): 역전파(Backpropagation)**

    손실 함수에 대한 모델의 가중치 기울기(Gradient) 계산


6. **optimizer.step(): 가중치 업데이트**
계산된 기울기를 기반으로 가중치(weight)와 편향(bias)를 업데이트

7. **print(f"Epoch [{epoch+1}/{num_epochs}]**,  **Loss: {running_loss/len(train_loader):.4f}"): 학습 진행 확인**
running_loss는 한 epoch 동안의 전체 손실을 저장 -> 평균 손실을 출력하여 모델이 학습하면서 손실이 점점 감소하는지 확인

# 모델 평가

In [28]:
model.eval()
correct = 0
total = 0

with torch.no_grad():  # 평가 시에는 기울기 계산 X
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"✅ 테스트 정확도: {accuracy:.2f}%")


✅ 테스트 정확도: 28.42%


# ✅ 추가 기능: 단일 이미지 예측
사용자가 직접 업로드한 이미지를 불러오기
→ PIL.Image.open()으로 로드
모델이 예측할 수 있도록 변환 적용
→ transforms를 사용해 Tensor 변환
모델을 활용해 분류
→ model(image_tensor.unsqueeze(0)) 형태로 예측
예측 결과 출력
→ 가장 높은 확률을 가진 클래스로 음식 분류

# 
📌 코드: 새로운 이미지 예측 함수

In [ ]:
# 현재 작업 폴더 확인 및 데이터 폴더 존재 여부 체크

print("현재 작업 폴더:", os.getcwd())  # 현재 위치 확인


현재 작업 디렉토리: /mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-6a025-ml/code/Users/6a025/cnn_test/data_set


In [ ]:
# 테스트 폴더 내 이미지 확인
test_folder = "./sample_data/test/김치찌개"
print("📂 폴더 내 파일 목록:", os.listdir(test_folder))


📂 폴더 내 파일 목록: ['.amlignore', '.amlignore.amltmp', 'Img_119_0214.jpg', 'Img_119_0215.jpg', 'Img_119_0275.jpg', 'Img_119_0288.jpg', 'Img_119_0296.jpg', 'sample_image.jpg']


In [ ]:
# 📌 올바른 이미지 파일 경로 설정 (예측할 이미지 지정)
image_path = "./sample_data/test/김치찌개/sample_image.jpg"  # 실제 이미지 경로 입력

# 📌 이미지 로드 및 전처리
image = Image.open(image_path)  # 이미지 파일 열기

# 📌 이미지 전처리 (모델 입력 크기 맞추기)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 모델 입력 크기와 동일하게 변환
    transforms.ToTensor(),  # PyTorch 텐서(Tensor)로 변환
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화 (-1~1 범위)
])
image = transform(image).unsqueeze(0)  # 배치 차원 추가 (모델 입력을 위해 필요)

# 📌 모델을 평가 모드로 설정
model.eval()

# 📌 모델 예측 수행
with torch.no_grad():  # 평가 시에는 기울기 계산 X
    output = model(image)  # 모델에 이미지 입력
    predicted_class = torch.argmax(output, 1).item()  # 가장 높은 확률의 클래스 선택

# 📌 클래스 이름 가져오기 (학습 데이터 폴더에서 클래스명 추출)
class_names = os.listdir("./sample_data/train")  # 학습 데이터 파일의 폴더명 가져오기

# 📌 예측 결과 출력
print(f"📌 예측 결과: {class_names[predicted_class]}")


📌 예측 결과: 김치찌개


📌 실행 흐름 정리

1️⃣ 이미지 로드 (PIL.Image.open())

2️⃣ 이미지 전처리 (Resize → ToTensor → Normalize → unsqueeze(0))

3️⃣ 모델을 평가 모드로 전환 (model.eval())

4️⃣ 모델 예측 수행 (torch.argmax(output, 1))

5️⃣ 학습 데이터에서 클래스명 가져오기 (os.listdir())

6️⃣ 최종 예측 결과 출력 (print(class_names[predicted_class])